In [1]:
FIGURES_PATH = 'out/figures/'
DATASETS_PATH = 'out/datasets/'

In [2]:
%pip install multiprocesspandas

Note: you may need to restart the kernel to use updated packages.


In [29]:
pip install -U numba

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.6/3.6 MB 3.1 MB/s eta 0:00:000:00:01m eta 0:00:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 42.1/42.1 MB 3.5 MB/s eta 0:00:00m eta 0:00:010:00:01
Note: you may need to restart the kernel to use updated packages.


In [30]:
import pandas as pd
from datetime import datetime, timedelta
import os
import multiprocessing
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.colors as mcolors
import random
# from tqdm import tqdm
from tqdm.notebook import tqdm
from multiprocesspandas import applyparallel
from pandarallel import pandarallel
import numba

tqdm.pandas()
from helper import *

In [4]:
NROWS = 250_000

In [5]:
data = pd.read_csv(DATASETS_PATH + 'data_processed.csv', nrows=NROWS).drop(columns=['Unnamed: 0'])
data['datetime'] = pd.to_datetime(data['datetime'])
# data

In [6]:
def get_user_purchases(data, batch_size):
    """
    :param data: receipts - pandas.DataFrame
    :return: ans: ans[i][j] = count of purchases by the user i of the product j - matrix
    """
    ans = np.zeros((data['gid'].drop_duplicates().shape[0], data['product_id'].drop_duplicates().shape[0]), dtype=np.int8)
    data = data[['gid', 'product_id']]

    def fill_arr(x, arr):
        name = x.name
        for i in x.values:
            arr[name][i] += 1

    def process_batch(batch):
        nonlocal ans
        batch.groupby(by='gid')['product_id'].apply(lambda x: fill_arr(x, ans))

    pool = multiprocessing.Pool(processes=multiprocessing.cpu_count())
    for batch in tqdm(np.array_split(data, data.shape[0] / batch_size)):
        batch = process_batch(batch)
        # pool.apply_async(
        #     process_batch,
        #     args=(batch,),
        # )
    pool.close()
    pool.join()

    return ans

In [7]:
# %pip install pandarallel

In [8]:
def get_date_distances(data):
    """
    Считаем по каждому пользователю ближайшие (по модулю даты) покупки товаров.
    Усредняем значения по каждому пользователю

    :param data:
    :return:
    """

    # def concat_ans(ans):
        # return np.sum(ans['timedelta']), ans.shape[0]

    def data_splitting(interval):
        nonlocal data
        batches = []
        data = data.sort_values(by='datetime')
        start = data.iloc[0].at['datetime']
        end = data.iloc[-1].at['datetime']
        while start <= end:
            sub_end = start + timedelta(days=interval)
            batch = data.loc[data['datetime'] >= start].loc[data['datetime'] < sub_end]
            batches.append(batch)
            start = sub_end

        return batches


    def do_dataframe(temps):
        # t = temps.values

        # print(t.shape)
        # t = np.concatenate(t)#np.concatenate(t))
        # print(t.shape)
        ans = pd.DataFrame(data=temps, columns=['product_1', 'product_2', 'timedelta'])
        ans['count'] = pd.Series(data=[1 for _ in range(ans.shape[0])])
        return ans


    def fill_ans(x):
        product_date = x[['product_id', 'datetime']]

        temp = []

        for i1, r1 in product_date.iterrows():
            for i2, r2 in product_date.iterrows():
                if i1 != i2:
                    p1, p2 = r1['product_id'], r2['product_id']
                    timedelta = (r1['datetime'] - r2['datetime']).days

                    is_found = False
                    for t in temp:
                        if t[0] == p1 and t[1] == p2:
                            if abs(timedelta) < abs(t[2]):
                                t[2] = timedelta
                                is_found = True

                    if not is_found:
                        temp.append([p1, p2, timedelta])

        if len(temp) != 0:
            return np.array(temp)

    data = data[['gid', 'product_id', 'datetime']]
    data.loc[:, 'datetime'] = data['datetime'].dt.date

    batches = data_splitting(interval=14)

    pandarallel.initialize(progress_bar=True, use_memory_fs=True)
    temps = []
    for batch in tqdm(batches):
        grouped_by_user = batch.groupby(by='gid')
#         temp = grouped_by_user.parallel_apply(fill_ans)
        temp = grouped_by_user.apply_parallel(fill_ans, num_processes=multiprocessing.cpu_count())
        temp = temp.dropna()
        temp = np.concatenate(np.concatenate(temp.values))


        temps.append(temp)

    temps = np.concatenate(temps)

    #
    # # print(temps)
    ans = do_dataframe(temps)
    #
    # del temps
    #
    ans.groupby(by=['product_1', 'product_2']).progress_apply(np.sum)#parallel_apply(np.sum)
#     ans.groupby(by=['product_1', 'product_2']).apply_parallel(np.sum, num_processes=multiprocessing.cpu_count())
    #
    return ans





In [9]:
dists = get_date_distances(data)

INFO: Pandarallel will run on 8 workers.
INFO: Pandarallel will use Memory file system to transfer data between the main process and workers.


  0%|          | 0/26 [00:00<?, ?it/s]


100%|█████████████████████████████████████| 2007/2007 [00:01<00:00, 1584.96it/s]

100%|█████████████████████████████████████| 2029/2029 [00:01<00:00, 1686.02it/s]

100%|█████████████████████████████████████| 1984/1984 [00:01<00:00, 1395.34it/s]

100%|█████████████████████████████████████| 2013/2013 [00:01<00:00, 1535.27it/s]

100%|█████████████████████████████████████| 1966/1966 [00:01<00:00, 1786.08it/s]

100%|█████████████████████████████████████| 2025/2025 [00:01<00:00, 1631.81it/s]

100%|█████████████████████████████████████| 2019/2019 [00:01<00:00, 1651.26it/s]

100%|█████████████████████████████████████| 2134/2134 [00:01<00:00, 1940.83it/s]

100%|█████████████████████████████████████| 2039/2039 [00:01<00:00, 1888.45it/s]

100%|█████████████████████████████████████| 2052/2052 [00:01<00:00, 1967.72it/s]

100%|█████████████████████████████████████| 2046/2046 [00:01<00:00, 2041.97it/s]

 79%|█████████████████████████████▍       | 1702/2142 [00:00<00:00, 2221.21it/s]


100%|█████████████████████████████████████| 2142/2142 [00:01<00:00, 1853.96it/s]

100%|█████████████████████████████████████| 1908/1908 [00:01<00:00, 1819.94it/s]

100%|█████████████████████████████████████| 1876/1876 [00:01<00:00, 1740.49it/s]

100%|█████████████████████████████████████| 2074/2074 [00:01<00:00, 1621.33it/s]

100%|█████████████████████████████████████| 2099/2099 [00:01<00:00, 1743.12it/s]

100%|█████████████████████████████████████| 1961/1961 [00:00<00:00, 2035.33it/s]

100%|█████████████████████████████████████| 2000/2000 [00:01<00:00, 1931.86it/s]

100%|█████████████████████████████████████| 1906/1906 [00:00<00:00, 1921.69it/s]

100%|█████████████████████████████████████| 1023/1023 [00:00<00:00, 1833.97it/s]

100%|█████████████████████████████████████| 1194/1194 [00:00<00:00, 1534.74it/s]

100%|█████████████████████████████████████| 1281/1281 [00:00<00:00, 1744.47it/s]

100%|█████████████████████████████████████| 1252/1252 [00:00<00:00, 1859.89it/s]

100%|███████████

  0%|          | 0/2166843 [00:00<?, ?it/s]

In [11]:
dists.sort_values(by='timedelta', ascending=False)

,product_1,product_2,timedelta,count
1440681,42349,1655,13,1
2490961,44458,4127,13,1
2490971,7449,4127,13,1
2490976,2985,4127,13,1
2490981,17660,4127,13,1
...,...,...,...,...
12482,10543,7646,-13,1
12483,10543,7645,-13,1
12492,567,7646,-13,1
12493,567,7645,-13,1


In [33]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 250000 entries, 0 to 249999
Data columns (total 11 columns):
 #   Column           Non-Null Count   Dtype         
---  ------           --------------   -----         
 0   gid              250000 non-null  int64         
 1   transaction_key  250000 non-null  int64         
 2   store_id         250000 non-null  int64         
 3   product_id       250000 non-null  int64         
 4   line_item_price  250000 non-null  float64       
 5   line_item_cost   233291 non-null  float64       
 6   line_type        250000 non-null  int64         
 7   datetime         250000 non-null  datetime64[ns]
 8   category_id      12 non-null      float64       
 9   weekday          250000 non-null  int64         
 10  line_quantity    250000 non-null  float64       
dtypes: datetime64[ns](1), float64(4), int64(6)
memory usage: 21.0 MB


In [42]:
def get_date_distances_map(data):
    """
    Считаем по каждому пользователю ближайшие (по модулю даты) покупки товаров.
    Усредняем значения по каждому пользователю

    :param data:
    :return:
    """

    # def concat_ans(ans):
        # return np.sum(ans['timedelta']), ans.shape[0]
        
    ans = dict()
    
    

    def data_splitting(interval):
        nonlocal data
        batches = []
        data = data.sort_values(by='datetime')
        start = data.iloc[0].at['datetime']
        end = data.iloc[-1].at['datetime']
        while start <= end:
            sub_end = start + timedelta(days=interval)
            batch = data.loc[data['datetime'] >= start].loc[data['datetime'] < sub_end]
            batches.append(batch)
            start = sub_end

        return batches


    def do_dataframe(temps):
        # t = temps.values

        # print(t.shape)
        # t = np.concatenate(t)#np.concatenate(t))
        # print(t.shape)
        ans = pd.DataFrame(data=temps, columns=['product_1', 'product_2', 'timedelta'])
        ans['count'] = pd.Series(data=[1 for _ in range(ans.shape[0])])
        return ans

    
    
    def fill_ans(x):
        product_date = x[['product_id', 'datetime']]

        temp = []

        for i1, r1 in product_date.iterrows():
            for i2, r2 in product_date.iterrows():
                if i1 != i2:
                    p1, p2 = r1['product_id'], r2['product_id']
                    timedelta = (r1['datetime'] - r2['datetime']).days

                    is_found = False
                    for t in temp:
                        if t[0] == p1 and t[1] == p2:
                            if abs(timedelta) < abs(t[2]):
                                t[2] = timedelta
                                is_found = True

                    if not is_found:
                        temp.append([p1, p2, timedelta])

        if len(temp) != 0:
            return np.array(temp)

    
    
    def concat_for_all_users(arrs):
        ans  = dict()
        for arr in arrs:
            if arr is not None:
                for one_arr in arr:
                    p1, p2, td = one_arr
                    if (p1, p2) in ans:
                        ans[(p1, p2)].append(td)
#                         ans[(p1, p2)] = (ans[(p1, p2)][0] + td, ans[(p1, p2)][1] + 1)
                    else:
                        ans[(p1, p2)] = [td]
#                         ans[(p1, p2)] = (td, 1)
                        
        return ans
    
    
#     def get_average(ans):
        
        
        

    data = data[['gid', 'product_id', 'datetime']]
    data.loc[:, 'datetime'] = data['datetime'].dt.date

    batches = data_splitting(interval=14)

    pandarallel.initialize(progress_bar=True, use_memory_fs=True)
    temps = []
    for batch in tqdm(batches):
        grouped_by_user = batch.groupby(by='gid')
        temp = grouped_by_user.parallel_apply(fill_ans)
#         temp = grouped_by_user.apply_parallel(fill_ans, num_processes=multiprocessing.cpu_count())
        temp = temp.dropna()
        temp = np.concatenate(np.concatenate(temp.values))


        temps.append(temp)
        
    ans = concat_for_all_users(temps)
    
    ans = pd.DataFrame(ans)

#     temps = np.concatenate(temps)

#     #
#     # # print(temps)
#     ans = do_dataframe(temps)
#     #
#     # del temps
#     #
#     ans.groupby(by=['product_1', 'product_2']).progress_apply(np.sum)#parallel_apply(np.sum)
# #     ans.groupby(by=['product_1', 'product_2']).apply_parallel(np.sum, num_processes=multiprocessing.cpu_count())
#     #
    return ans





In [43]:
dists_map = get_date_distances_map(data)

INFO: Pandarallel will run on 8 workers.
INFO: Pandarallel will use Memory file system to transfer data between the main process and workers.


  0%|          | 0/26 [00:00<?, ?it/s]


100%|█████████████████████████████████████| 2007/2007 [00:00<00:00, 2576.45it/s]

100%|█████████████████████████████████████| 2029/2029 [00:01<00:00, 1869.58it/s]

100%|█████████████████████████████████████| 1984/1984 [00:01<00:00, 1584.65it/s]

100%|█████████████████████████████████████| 2013/2013 [00:01<00:00, 1915.38it/s]

100%|█████████████████████████████████████| 1966/1966 [00:01<00:00, 1577.67it/s]

100%|█████████████████████████████████████| 2025/2025 [00:01<00:00, 1603.84it/s]

100%|█████████████████████████████████████| 2019/2019 [00:01<00:00, 1912.26it/s]

100%|█████████████████████████████████████| 2134/2134 [00:01<00:00, 1878.78it/s]

100%|█████████████████████████████████████| 2039/2039 [00:01<00:00, 1764.24it/s]

100%|█████████████████████████████████████| 2052/2052 [00:01<00:00, 1788.59it/s]

100%|█████████████████████████████████████| 2046/2046 [00:01<00:00, 1601.09it/s]

 68%|█████████████████████████▉            | 1463/2142 [00:01<00:00, 927.18it/s]


100%|██████████████████████████████████████| 2142/2142 [00:02<00:00, 994.61it/s]

100%|█████████████████████████████████████| 1908/1908 [00:01<00:00, 1198.10it/s]

100%|█████████████████████████████████████| 1876/1876 [00:01<00:00, 1405.62it/s]

100%|██████████████████████████████████████| 2074/2074 [00:02<00:00, 841.65it/s]

100%|█████████████████████████████████████| 2099/2099 [00:01<00:00, 1173.81it/s]

100%|██████████████████████████████████████| 1961/1961 [00:02<00:00, 942.25it/s]

100%|█████████████████████████████████████| 2000/2000 [00:01<00:00, 1048.04it/s]

100%|██████████████████████████████████████| 1906/1906 [00:02<00:00, 842.06it/s]

 55%|█████████████████████▌                 | 567/1023 [00:00<00:00, 957.88it/s]


100%|██████████████████████████████████████| 1023/1023 [00:01<00:00, 988.14it/s]

100%|█████████████████████████████████████| 1194/1194 [00:00<00:00, 1456.62it/s]

100%|█████████████████████████████████████| 1281/1281 [00:00<00:00, 1425.39it/s]

100%|█████████████████████████████████████| 1252/1252 [00:00<00:00, 1545.77it/s]

100%|█████████████████████████████████████| 1249/1249 [00:00<00:00, 1317.29it/s]

100%|██████████████████████████████████████| 1350/1350 [00:01<00:00, 965.22it/s]

100%|█████████████████████████████████████| 1452/1452 [00:00<00:00, 1551.02it/s]


ValueError: All arrays must be of the same length

In [ ]:
dists_map